In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords 
from nltk.tokenize import WordPunctTokenizer

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
import warnings
warnings.filterwarnings('ignore')

In [0]:
df = pd.read_csv(r'yelp_review_arizona.csv')
df_business = pd.read_csv(r'yelp_business.csv')
df.head()

,review_id,user_id,business_id,text,stars,date
0,V93SYj2OLh5m9Cquzf-7kg,ZwVz20be-hOZnyAbevyMyQ,2c9Vptks_vowLgVUMnCgjw,Came here while in town for a country concert....,4.0,2013-09-04 01:29:46
1,vNTFadc6T9HeH3Qa78dc_Q,91TB-gzcNyxFh46TL0pmnQ,6nKR80xEGHYf2UxAe_Cu_g,Best barbecue this side of the Mississippi!!!!...,5.0,2015-12-05 02:50:10
2,SXRFBCt5eXCBF7TlI7UG6Q,Y_QBiZpATJoz8hKUfYF66A,fbQaKW0Lte0JQ_opbnjdKg,Absolutely amazing. Think Chipotle for enchila...,5.0,2014-04-01 01:56:00
3,CqMNjtG0hNZGhDw4RDE-zw,_Jg-IA0M-GSjBlGu-wmejg,r8764MtYyt8JhxMvrfM_xQ,I was really disappointed with my most recent ...,2.0,2014-10-11 03:53:53
4,5hZLouGEW4wm6BTJ5aNUNw,1CqkFliipv_X15WYn5aPfg,QS3QxI7u5PRdtbGgI0-UsA,I grade sushi restaurants on 3 factors:\n- Qua...,4.0,2015-03-04 19:36:21


In [0]:
#Select only stars and text
yelp_data = df[['business_id', 'user_id', 'stars', 'text']]

In [0]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
import string
from nltk.corpus import stopwords
stop = []
for word in stopwords.words('english'):
    s = [char for char in word if char not in string.punctuation]
    stop.append(''.join(s))

In [0]:
def text_process(mess):
    """
    Takes in a string of text, then performs the following:
    1. Remove all punctuation
    2. Remove all stopwords
    3. Returns a list of the cleaned text
    """
    # Check characters to see if they are in punctuation
    nopunc = [char for char in mess if char not in string.punctuation]

    # Join the characters again to form the string.
    nopunc = ''.join(nopunc)
    
    # Now just remove any stopwords
    return " ".join([word for word in nopunc.split() if word.lower() not in stop])
yelp_data['text'] = yelp_data['text'].apply(text_process)

In [0]:
userid_df = yelp_data[['user_id','text']]
business_df = yelp_data[['business_id', 'text']]

In [0]:
userid_df[userid_df['user_id']=='ZwVz20be-hOZnyAbevyMyQ']['text']

0        Came town country concert better way start day...
1098     Amazing Im golfer ended loving went family lov...
3909     saw many great reviews place decided try high ...
11491    Amazing Mexican Asian infusion good service sa...
19647    beautiful hotel excellent service Nick front a...
Name: text, dtype: object

In [0]:
userid_df = userid_df.groupby('user_id').agg({'text': ' '.join})
business_df = business_df.groupby('business_id').agg({'text': ' '.join})

In [0]:
userid_df.loc['ZwVz20be-hOZnyAbevyMyQ']['text']

'Came town country concert better way start day Everything great service amazing time walked door sat table surprise beer whiskey tap table pay ounce screen show drank food great whiskey burger favorite country fans must Scottsdale Amazing Im golfer ended loving went family loves golf planned staying hour since golf 6 hours later left far drive place right food amazing expect good brisket tacos chorizo sliders favorite Good quality food many unique items highly recommend place people ages teens loved bays nice roomy 7 us bay 10 people must try back soon saw many great reviews place decided try high hopes loved atmosphere kids loved Texas trivia game thats good thing could say steak came cooked way asked took back fix steak flavor covered butter thought steak even kids eat theres waitress asked daughter something wrong since ate bites explained like said ok walked away shrimp good pricey bugers ok end pushing us take home since ate almost none politely declined still came boxes trying g

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [0]:
#userid vectorizer
userid_vectorizer = TfidfVectorizer(tokenizer = WordPunctTokenizer().tokenize, max_features=5000)
userid_vectors = userid_vectorizer.fit_transform(userid_df['text'])
userid_vectors.shape

(10937, 5000)

In [0]:
#Business id vectorizer
businessid_vectorizer = TfidfVectorizer(tokenizer = WordPunctTokenizer().tokenize, max_features=5000)
businessid_vectors = businessid_vectorizer.fit_transform(business_df['text'])
businessid_vectors.shape

(1411, 5000)

In [0]:
userid_rating_matrix = pd.pivot_table(yelp_data, values='stars', index=['user_id'], columns=['business_id'])
userid_rating_matrix.shape

(10937, 1411)

In [0]:

P = pd.DataFrame(userid_vectors.toarray(), index=userid_df.index, columns=userid_vectorizer.get_feature_names())
Q = pd.DataFrame(businessid_vectors.toarray(), index=business_df.index, columns=businessid_vectorizer.get_feature_names())

In [0]:
def matrix_factorization(R, P, Q, steps=25, gamma=0.001,lamda=0.02):
    for step in range(steps):
        for i in R.index:
            for j in R.columns:
                if R.loc[i,j]>0:
                    eij=R.loc[i,j]-np.dot(P.loc[i],Q.loc[j])
                    P.loc[i]=P.loc[i]+gamma*(eij*Q.loc[j]-lamda*P.loc[i])
                    Q.loc[j]=Q.loc[j]+gamma*(eij*P.loc[i]-lamda*Q.loc[j])
        e=0
        for i in R.index:
            for j in R.columns:
                if R.loc[i,j]>0:
                    e= e + pow(R.loc[i,j]-np.dot(P.loc[i],Q.loc[j]),2)+lamda*(pow(np.linalg.norm(P.loc[i]),2)+pow(np.linalg.norm(Q.loc[j]),2))
        if e<0.001:
            break
        
    return P,Q

In [0]:
%%time
P, Q = matrix_factorization(userid_rating_matrix, P, Q, steps=25, gamma=0.001,lamda=0.02)

CPU times: user 3h 8min 47s, sys: 2.89 s, total: 3h 8min 50s
Wall time: 3h 9min 19s


In [0]:
Q.iloc[0].sort_values(ascending=False).head(10)

matts        1.077482
breakfast    0.659543
wait         0.529304
food         0.428016
place        0.405557
eggs         0.362205
good         0.354656
bacon        0.312531
like         0.274301
big          0.265078
Name: -050d_XIor1NpCuWkbIVaQ, dtype: float64

In [0]:
# Store P, Q and vectorizer in pickle file
import pickle
output = open('yelp_recommendation_model_8.pkl', 'wb')
pickle.dump(P,output)
pickle.dump(Q,output)
pickle.dump(userid_vectorizer,output)
output.close()

In [0]:
#static input
words = "i want to have dinner with excellent dessert"
test_df= pd.DataFrame([words], columns=['text'])
test_df['text'] = test_df['text'].apply(text_process)
test_vectors = userid_vectorizer.transform(test_df['text'])
test_v_df = pd.DataFrame(test_vectors.toarray(), index=test_df.index, columns=userid_vectorizer.get_feature_names())

predictItemRating=pd.DataFrame(np.dot(test_v_df.loc[0],Q.T),index=Q.index,columns=['Rating'])
topRecommendations=pd.DataFrame.sort_values(predictItemRating,['Rating'],ascending=[0])[:7]

for i in topRecommendations.index:
    print(df_business[df_business['business_id']==i]['name'].iloc[0])
    print(df_business[df_business['business_id']==i]['categories'].iloc[0])
    print(str(df_business[df_business['business_id']==i]['stars'].iloc[0])+ ' '+str(df_business[df_business['business_id']==i]['review_count'].iloc[0]))
    print('')

Steak 44
Restaurants, Steakhouses, Bars, Nightlife, Wine Bars, Seafood
4.5 950

Citizen Public House
Cocktail Bars, Bars, Nightlife, American (New), Local Flavor, Gastropubs, Salad, Restaurants
4.5 1797

The White Chocolate Grill
Food, American (Traditional), Desserts, Restaurants
4.0 975

Roaring Fork
Nightlife, Desserts, Steakhouses, Food, Restaurants, Bars, American (Traditional)
4.0 1072

The Parlor
Salad, Italian, Sandwiches, Pizza, Restaurants
4.5 1199

Cibo
Restaurants, Italian, Sandwiches, Pizza
4.5 1955

Rusconi's American Kitchen
Restaurants, American (Traditional), Breakfast & Brunch, American (New)
4.5 624



In [0]:
#dynamic input
words = input()
test_df= pd.DataFrame([words], columns=['text'])
test_df['text'] = test_df['text'].apply(text_process)
test_vectors = userid_vectorizer.transform(test_df['text'])
test_v_df = pd.DataFrame(test_vectors.toarray(), index=test_df.index, columns=userid_vectorizer.get_feature_names())
predictItemRating=pd.DataFrame(np.dot(test_v_df.loc[0],Q.T),index=Q.index,columns=['Rating'])
topRecommendations=pd.DataFrame.sort_values(predictItemRating,['Rating'],ascending=[0])[:7]
for i in topRecommendations.index:
    print(df_business[df_business['business_id']==i]['name'].iloc[0])
    print(df_business[df_business['business_id']==i]['categories'].iloc[0])
    print(str(df_business[df_business['business_id']==i]['stars'].iloc[0])+ ' '+str(df_business[df_business['business_id']==i]['review_count'].iloc[0]))
    print('')

i want to have a delightful dinner
Lux Central
Bakeries, American (New), Nightlife, Bars, Coffee & Tea, Restaurants, Breakfast & Brunch, Food
4.5 2046

Citizen Public House
Cocktail Bars, Bars, Nightlife, American (New), Local Flavor, Gastropubs, Salad, Restaurants
4.5 1797

Windsor
Restaurants, Nightlife, Bars, Breakfast & Brunch, Pubs, Food, American (New), Beer, Wine & Spirits
4.0 1100

Steak 44
Restaurants, Steakhouses, Bars, Nightlife, Wine Bars, Seafood
4.5 950

Roaring Fork
Nightlife, Desserts, Steakhouses, Food, Restaurants, Bars, American (Traditional)
4.0 1072

Chelsea's Kitchen
Breakfast & Brunch, Restaurants, American (New), American (Traditional)
4.0 1363

The Henry
Salad, Venues & Event Spaces, Event Planning & Services, American (New), Breakfast & Brunch, Restaurants
4.0 1176

